In [ ]:
print("Python běží!")

In [ ]:
import sys

sys.path.append('../')
sys.path.append('../my_libs/')
sys.path.append('../my_libs/img/')

!{sys.executable} -m pip install -r ../../pip-dependencies.txt

In [ ]:
import my_libs as ml
from my_libs.tools import *
from my_libs.fft import *
from my_libs.colors import *
from my_libs.img.processing import *

In [ ]:
ml.tools.printf("Hello world!")

# Cvičení 6
1) Načtěte obrázek pvi_cv06_mince.jpg. Obraz převeďte do složky Hue. Na 
základě  histogramu  stanovte  práh  a  obraz  segmentujte.  V  jednom  okně 
zobrazte RGB a Hue obraz, histogram a výsledný segmentovaný obraz. 
 
2)  Na  výsledný  binární  obraz  aplikujte  algoritmus  rozvodí  (watershed). 
Nalezenou  hranici  z  rozvodí  zvětšete  pomocí  dilatace  se  strukturním 
elementem 3x3 a odstraňte z původního segmentovaného obrazu, aby došlo 
k oddělení objektů.  
 
3) Artefakty z oddělení objektů odsraňte při využítí algoritmu barvení oblastí. 
Např.: všechny objekty, které mají méně než 1000 obrazových bodů budou 
odstraněny. 
 
4)  Z  binárního  obrazu  vytvořte  granulometrické  spektrum.  Spektrum 
analyzujte  (např.  od  objektů  velikosti  40x40  obrazových  bodů).  Všechny 
poměry menší než 0,9 zanedbejte. Zaokrouhlení poměru - bez desetiné části.  
 
5)  Výstupem bude velikost objektu a počet objektů. 
 
help:  
- https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html 